<a href="https://colab.research.google.com/github/amimulhasan/Deep-Learning/blob/main/Image_Classification_using_BigTransfer_(BiT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.18.0


In [2]:
# Hyperparameters
batch_size = 32
img_size = (224, 224)
num_classes = 5  # Adjust as needed
epochs = 10


In [3]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle datasets download shuvokumarbasakbd/brain-ct-medical-imaging-colorized-dataset

Dataset URL: https://www.kaggle.com/datasets/shuvokumarbasakbd/brain-ct-medical-imaging-colorized-dataset
License(s): MIT
 84% 78.0M/93.1M [00:00<00:00, 816MB/s]
100% 93.1M/93.1M [00:00<00:00, 786MB/s]


In [5]:
import zipfile

zip_path = '/content/brain-ct-medical-imaging-colorized-dataset.zip'
extract_to = 'brain_CT'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Unzipping completed!")

Unzipping completed!


In [8]:
data_dir = '/content/brain_CT/Computed Tomography (CT) of the Brain/dataset/train'  # or your actual path

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=img_size,
    batch_size=batch_size,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=img_size,
    batch_size=batch_size,
)

# ⚠️ Get class names BEFORE prefetch
num_classes = len(train_ds.class_names)
print("Detected classes:", train_ds.class_names)

# Now prefetch
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)


Found 2590 files belonging to 3 classes.
Using 2072 files for training.
Found 2590 files belonging to 3 classes.
Using 518 files for validation.
Detected classes: ['aneurysm', 'cancer', 'tumor']


In [9]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
    ]
)


In [11]:
base_model = keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=img_size + (3,),
)

base_model.trainable = False


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [14]:
inputs = keras.Input(shape=img_size + (3,))
x = data_augmentation(inputs)
x = keras.applications.resnet.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(num_classes)(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 224, 224,  │          0 │ input_layer_3[0]… │
│ (Sequential)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 224, 224)  │          0 │ sequential[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 224, 224)  │          0 │ sequential[1][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 224, 224)  │          0 │ sequential[1][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack (Stack)       │ (None, 224, 224,  │          0 │ get_item[0][0],   │
│                     │ 3)                │            │ get_item_1[0][0], │
│                     │                   │            │ get_item_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 224, 224,  │          0 │ stack[0][0]       │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 7, 7,      │ 23,587,712 │ add[0][0]         │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2048)      │          0 │ resnet50[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 3)         │      6,147 │ global_average_p… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 23,593,859 (90.00 MB)

 Trainable params: 6,147 (24.01 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
)


Epoch 1/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 500s 8s/step - accuracy: 0.6780 - loss: 0.7381 - val_accuracy: 0.9865 - val_loss: 0.1003
Epoch 2/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 495s 7s/step - accuracy: 0.9899 - loss: 0.0777 - val_accuracy: 0.9923 - val_loss: 0.0521
Epoch 3/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 502s 7s/step - accuracy: 0.9888 - loss: 0.0515 - val_accuracy: 0.9942 - val_loss: 0.0355
Epoch 4/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 442s 6s/step - accuracy: 0.9929 - loss: 0.0347 - val_accuracy: 0.9942 - val_loss: 0.0302
Epoch 5/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 426s 7s/step - accuracy: 0.9946 - loss: 0.0311 - val_accuracy: 0.9942 - val_loss: 0.0249
Epoch 6/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 439s 7s/step - accuracy: 0.9954 - loss: 0.0276 - val_accuracy: 0.9942 - val_loss: 0.0228
Epoch 7/10
38/65 ━━━━━━━━━━━━━━━━━━━━ 2:22 5s/step - accuracy: 0.9974 - loss: 0.0206

In [ ]:
base_model.trainable = True  # Unfreeze the base model for fine-tuning

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # lower LR for fine-tuning
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

history_finetune = model.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds,
)
